# Create Figures

## $\theta_0$ & $\theta_{90}$ vs inclination

This figure attempts to obtain $\theta_0$ and $\theta_{90}$ as a function of inclination for quadrics section. I plan to do a reference in section 4.3

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import fsolve
from scipy.special import gamma as gamma_func

Equations to solve:
\begin{align}
T(\theta_0) &= \cot^2\theta_c\cot i + \frac{x_0}{b}\sqrt{\cot^2\theta_c\cot^2 i \pm 1}\\
T(\theta_{90}) &= f_2(i;\theta_c)\left[\frac{x_0}{b} - \frac{x_0}{a}\frac{|\cot\theta_c|}{f^2(i;\theta_c)}\right] \\
\frac{x_0}{b} &= \frac{|\tan\theta_c|}{\tilde{R}_c} - |\cot\theta_c|\\
\frac{x_0}{a} &= \frac{\tan^2\theta_c}{\tilde{R}_c} \mp 1 \\
f(i;\theta_c) &= \left(1 \pm \tan^2 i\tan^2\theta_c\right)^{1/2}\\
f_2(i;\theta_c) &= \left[1 \mp \left(\frac{x_0}{a}\right)^2\frac{1}{f^4(i;\theta_c)}\right]^{-1/2}
\end{align}

In [8]:
def f(i, tc):
    """
    Function of inclination and theta_c
    """
    return np.sqrt(1 + np.sign(tc)*np.tan(np.radians(i))**2*np.tan(np.radians(tc))**2)

In [9]:
def x0a(Rc, tc):
    """
    x_0 over a ratio
    """
    return np.tan(np.radians(tc))**2/Rc -np.sign(tc)

In [10]:
def x0b(Rc, tc):
    """
    x_0 over b ratio
    """
    return np.abs(np.tan(np.radians(tc)))/Rc - 1./np.abs(np.tan(np.radians(tc)))

In [11]:
def f2(i, tc, Rc):
    """
    Function of inclination, theta_c and R_c
    """
    return np.sqrt(1 - np.sign(tc) + x0a(Rc, tc)**2/f(i, tc)**4)

Set parameters:

In [12]:
inclination = np.linspace(0,30)
beta = [1e-4, 1e-2, 1e-1]
xi = ["isotropic", 1, 0.8, 0.4]

Set more functions for the tail fit:

In [14]:
def x0_tail(beta, xi):
    """
    Hyperbola offset for tail
    """
    ck0 = [2.0758, -0.2309, -0.2532]
    ck1 = [0.9571, -0.1530, -0.2487]
    ck2 = [0.2528, -0.0360, -0.0794]
    ck3 = [0.0171, -0.0010, -0.0095]
    C3 =  ck3[2]*xi**2 + ck3[1]*xi + ck3[0]
    C2 =  ck2[2]*xi**2 + ck2[1]*xi + ck2[0]
    C1 =  ck1[2]*xi**2 + ck1[1]*xi + ck1[0]
    C0 =  ck0[2]*xi**2 + ck0[1]*xi + ck0[0]
    logb = np.log10(beta)
    return 0.7*beta**(-0.55)*(C3*logb**3 + C2*logb**2 + C1*logb + C0)

In [1]:
def x0ma_tail(beta, xi):
    """
    Find x0 - a for the tail fit
    """
    dk0 = [0.8516, -0.0907, -0.2002]
    dk1 = [-0.7620, 0.1411, -0.0295]
    dk2 = [-0.0683, 0.0390, -0.0236]
    D2 =  dk2[2]*xi**2 + dk2[1]*xi + dk2[0]
    D1 =  dk1[2]*xi**2 + dk1[1]*xi + dk1[0]
    D0 =  dk0[2]*xi**2 + dk0[1]*xi + dk0[0]
    logb = np.log10(beta)
    return D2*logb**2 + D1*logb + D0

## $\theta_\infty$ for non isotropic case

For the non isotropic case the inner wind mass loss truncates at $\theta = \frac{\pi}{2}$. So, the equation (54) of
paper does not apply for $\theta > \frac{\pi}{2}$ if $k > 0$. Instead, we got that the equivalent of equation (54) is:
\begin{align}
\theta_1\cot\theta_1 &= 2\beta I \cot\theta -\frac{2\beta}{k+2} \\
\mathrm{where~}  I &= \int^{\frac{\pi}{2}}_{0} \cos^k\theta\sin^2\theta~d\theta = \frac{\sqrt{\pi}\Gamma\left(\frac{k+1}{2}\right)}{4\Gamma\left(\frac{k}{2}+2\right)}
\end{align}

With this we can obtain $\theta_\infty$ defined as $\theta_\infty + \theta_{1\infty} = \pi$ solving the implicit equation for $\theta_\infty$:

$$\theta_\infty - \frac{k+2(1-\beta)}{k+2}\tan\theta_\infty = \pi + 2\beta I$$

In [3]:
def finf(th, beta, xi):
    k = 2./xi-2
    C = (k+2*(1-beta))/(k+2)
    I = np.sqrt(np.pi)*gamma_func(0.5*(k+1))/(4*gamma_func(0.5*k+2))
    D = np.pi + 2*beta*I
    return th - C*np.tan(th) -D

In [4]:
def finf_CRW(th, beta):
    return th - np.tan(th) - np.pi/(1.0 -beta)